<a href="https://colab.research.google.com/github/mariasuescum/G3-COMPUTER-VISION/blob/feat%2Fnotebook-training/model_training_medium_pre_decompressed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento de YOLOv8 en Google Colab (Dataset Pre-descomprimido)

Este notebook está adaptado para entrenar un modelo YOLOv8, asumiendo que el dataset ya está descomprimido en Google Drive. Guarda los resultados en Google Drive y tiene capacidad de reanudar el entrenamiento si la sesión se interrumpe.

### 1. Montar Google Drive

Conecta el notebook a tu Google Drive. Aquí es donde leeremos el dataset y guardaremos los resultados y checkpoints.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2. Instalar Librerías

Instalamos `ultralytics` para YOLOv8.

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.7 MB/s eta 0:00:00


### 3. Configuración del Entrenamiento

**ACCIÓN REQUERIDA:** Ajusta las rutas y parámetros en la siguiente celda según tu configuración.

In [4]:
import os
from ultralytics import YOLO

# --- PARÁMETROS DE CONFIGURACIÓN ---

# Asegurarse de que CUDA_VISIBLE_DEVICES esté configurado para usar la primera GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Ruta base en tu Google Drive donde tienes el proyecto
# DEBES AJUSTAR ESTA RUTA a donde hayas guardado tu dataset
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision' # Asegúrate de que esta ruta sea correcta

# --- Configuración del Dataset (ya descomprimido) ---
# Ruta a la carpeta del dataset ya descomprimido en Drive
DATASET_BASE_PATH = os.path.join(DRIVE_PROJECT_PATH, 'dataset_v1_yolov8_reduced')

# Ruta al archivo YAML del dataset
DATASET_YAML_PATH = os.path.join(DATASET_BASE_PATH, 'data.yaml')

# Modelo, epochs y nombre del run
MODEL_NAME = 'yolov8m.pt'
EPOCHS = 80
RUN_NAME = 'yolov8m_colab_80_epochs_pre_decompressed'

# Parámetros de rendimiento para Colab
BATCH_SIZE = 8
IMG_SIZE = 640
WORKERS = 4

# --- FIN DE LA CONFIGURACIÓN ---

# Asegurarse de que la carpeta de resultados exista en Drive
os.makedirs(os.path.join(DRIVE_PROJECT_PATH, 'training_results'), exist_ok=True)

# --- Verificación del Dataset ---
if not os.path.exists(DATASET_YAML_PATH):
    print(f"ERROR: El archivo data.yaml no se encontró en {DATASET_YAML_PATH}. Asegúrate de que el dataset esté descomprimido en la ruta correcta.")
else:
    print(f"Dataset listo en: {DATASET_BASE_PATH}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Dataset listo en: /content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision/dataset_v1_yolov8_reduced


### 4. Entrenar el Modelo

Esta celda inicia o reanuda el entrenamiento. Gracias a `resume=True`, si el script se detiene, puedes volver a ejecutar esta celda y continuará donde lo dejó.

In [5]:
from ultralytics import YOLO

# Cargar el modelo base (YOLO descargará 'yolov8m.pt' si no existe)
model = YOLO(MODEL_NAME)

# Iniciar el entrenamiento
results = model.train(
    # --- Rutas y Datos ---
    data=DATASET_YAML_PATH,
    project=os.path.join(DRIVE_PROJECT_PATH, 'training_results'), # Guarda los resultados en Drive
    name=RUN_NAME,

    # --- Parámetros de Entrenamiento ---
    epochs=EPOCHS,
    # resume=True,  # <-- ¡LA CLAVE! Reanuda el entrenamiento si encuentra un checkpoint

    # --- Parámetros de Rendimiento ---
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    workers=WORKERS,

    # --- Opcionales ---
    save_json=True,
    save_hybrid=True,
    plots=True,
    device='0' # Forzar el uso de la GPU 0
)

print('--- ENTRENAMIENTO FINALIZADO ---')
print(f'Resultados guardados en: {results.save_dir}')

WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in in the future.
Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision/dataset_v1_yolov8_reduced/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=Fal

### 5. Evaluar el Modelo Final

Una vez finalizado el entrenamiento completo, puedes ejecutar esta celda para ver las métricas de validación del mejor modelo.

In [6]:
# Cargar el mejor modelo guardado durante el entrenamiento
best_model_path = os.path.join(DRIVE_PROJECT_PATH, 'training_results', RUN_NAME, 'weights', 'best.pt')
model = YOLO(MODEL_NAME)

# Evaluar en el conjunto de validación
metrics = model.val()

print('--- MÉTRICAS DE VALIDACIÓN ---')
print(f'mAP50-95: {metrics.box.map:.4f}')
print(f'   mAP50: {metrics.box.map50:.4f}')
print(f'   mAP75: {metrics.box.map75:.4f}')

Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8m summary (fused): 92 layers, 25,886,080 parameters, 0 gradients, 78.9 GFLOPs

WARNING ⚠️ Dataset 'coco.yaml' images not found, missing path '/content/datasets/coco/val2017.txt'
Unzipping /content/datasets/coco2017labels-segments.zip to /content/datasets/coco...: 100% ━━━━━━━━━━━━ 122232/122232 6085.8files/s 20.1s
Unzipping /content/datasets/coco/images/val2017.zip to /content/datasets/coco/images/val2017...: 100% ━━━━━━━━━━━━ 5001/5001 383.8files/s 13.0s
Unzipping /content/datasets/coco/images/test2017.zip to /content/datasets/coco/images/test2017...: 100% ━━━━━━━━━━━━ 40671/40671 428.2files/s 1:35
Unzipping /content/datasets/coco/images/train2017.zip to /content/datasets/coco/images/train2017...: 100% ━━━━━━━━━━━━ 118288/118288 452.7files/s 4:21
Dataset download success ✅ (764.0s), saved to /content/datasets

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 67.8±22.9 MB/s, size: 185.8 KB)
val: S